In [1]:
import pandas as pd
import numpy as np
import copy
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [51]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48097 entries, 0 to 48096
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       48097 non-null  float64
 1   1       48097 non-null  float64
 2   2       48097 non-null  float64
 3   3       48097 non-null  float64
 4   4       48097 non-null  float64
 5   label   48097 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 2.2 MB


In [50]:
# データ取得
Data = pd.read_csv(r'C:\Users\0208071\Downloads\_ABEJA_dataset_interview+.csv')
# データセット情報確認（Null存在かどうかなど）
Data.info()
# 相関性確認。強い相関を持つ説明変数のどちらか除外する
Data.corr()
# 基本統計量など確認し、データ性質を把握する
Info = []
for i in range(5):
    for j in range(8):
      Info += [[i,j] + list(Data.loc[Data.label==j,Data.columns[i]].describe())]
Info = pd.DataFrame(Info,columns=['ColNum','label','count','mean','std','min','25%','50%','75%','max'])
# 今回のデータでは強い相関性確認されなかったため、除外なしでよい！
### ↑精度向上施策Ⅰ：相関性確認

In [40]:
# 特徴量
X = np.array(Data[Data.columns[0:5]])
# ラベル
y = np.array(Data[['label']]).reshape(-1, 1)
# 訓練データとテストデータに分割(訓練8:テスト2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 特徴量の標準化
# 訓練データの標準化時、テストデータをつかってはならない
### ↑精度向上施策Ⅱ：特徴量ごとに平均・標準偏差を使って標準化せずに、特徴量行列に平均ベクトル・共分散行列を使って、標準化する
### 相関性が低いが、相関性なしでないことを注意！
### 今回実施なし
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
# ラベルをワンホットエンコーディング
encoder = OneHotEncoder(sparse_output=False)
y_train_onehot = encoder.fit_transform(y_train)
y_test_onehot = encoder.fit_transform(y_test)

In [44]:
# モデル構築
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # 入力レイヤー
    Dense(256, activation='leaky_relu'), ### 精度向上施策Ⅲ：中間層数・クラス数・活性化関数を調整
    Dense(64, activation='leaky_relu'),
    Dense(y_train_onehot.shape[1], activation='softmax')
])
# モデルのコンパイル
model.compile(optimizer='adam', ### 精度向上施策Ⅳ：最適化関数種類・ロス関数種類を調整
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
# モデルの学習
history = model.fit(X_train, y_train_onehot, epochs=20, batch_size=32, verbose=1)
### 精度向上施策Ⅴ：訓練回数（epochs）・バッチサイズを調整
### 今回精度が安定的収束したため、訓練回数をこれ以上増やしても向上効果が出ない

Epoch 1/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7204 - loss: 0.8377
Epoch 2/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7964 - loss: 0.5630
Epoch 3/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8054 - loss: 0.5375
Epoch 4/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8118 - loss: 0.5232
Epoch 5/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8179 - loss: 0.5103
Epoch 6/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8152 - loss: 0.5131
Epoch 7/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8161 - loss: 0.5049
Epoch 8/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8204 - loss: 0.4953
Epoch 9/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8232 - loss: 0.4921
Epoch 10/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8183 - loss: 0.4978
Epoch 11/20
1203/1203 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8195 - loss: 0.4966
Epoch 12/20
1203/1203 ━━━━━━━━

In [49]:
# 評価
### 評価方法には、一般的に　⓵ROC曲線とAUC　⓶F1スコアという総合的な評価基準があげられる
### 分析目的によって、再現率に偏重したり、適合率に偏重したりすることはよくあるが、それらは根本的にモデル精度を向上することができない
### 個人的に、もっともシンプルの正解率を重視する。正解率が安定すれば、モデル精度も安定するのである
### モデル構築時に精度が高く、そのあと低下していくことは、よく発生する。安定しないモデルは信頼を失ってしまうことに！
loss, accuracy = model.evaluate(X_test, y_test_onehot, verbose=0)
print(f"テストデータの精度: {accuracy}")

テストデータの精度: 0.8255717158317566
